# Wild Fires caused by the weather
## Part 2: Data Cleaning

At this part, we will clean the data we crawled in the previous section. <br>
As a part of the data cleaning we will perform: <br>
• Columns cleaning <br>
• Missing data handling <br>
• Duplicate data handling <br>
• Data type conversion <br>
• Outliers handling

#### Preceding Step - import modules (packages)
This step is necessary in order to use external packages. 

In [1]:
import pandas as pd
from datetime import date as dt
import numpy as np

# Uncomment the below line for installing geopy for the first time
#!pip install geopy
from geopy.geocoders import Nominatim

#### Global variables and constants
Here we define our global variables we will use in this notebook

In [2]:
## CSV files 
BASE_CSV = "Wildfire_history_full.csv" # This is the unhandled csv file
CSV_COL_CLEAN = "Wildfire_history_col_clean.csv" # This is the file after columns cleaning
CSV_MISSING_CLEAN = "Wildfire_history_missing_data_clean.csv" # This is the file after missing data handling
CSV_DUP_CLEAN = "Wildfire_history_dup_data_clean.csv" # This is the file after duplicate data handling
CSV_DATA_CONVERSION = "Wildfire_history_data_conversion.csv" # This is the file after outliers handling
CSV_HANDLE_OUTLIERS = "Wildfire_history_outliers_handling.csv" # This is the file after data type conversion and all other cleanings

## Others
COLS = ['UniqueFireIdentifier', 'FireDiscoveryDateTime', 'FireOutDateTime', 'POOCounty', 'InitialLatitude', 'InitialLongitude', 'FireCause']

### Columns cleaning
In this section, we will remove all necessary columns and keep only the relevant ones: <br>
• Unique Fire Identifier <br>
• Fire Discovery Date Time <br>
• Fire Out Date Time <br>
• County <br>
• Latitude <br>
• Longitude <br>
• Fire Cause <br>
The columns related to the weather will be added after the data cleaning part(from weather API)

In [3]:
## This function is getting data frame, column names and new file name.
## It created a data frame with specific columns and saves it to a csv file
def removeColumns(df, column_names, file_name):
    cols=[c for c in column_names]
    new_df=df[cols]
    new_df.to_csv(file_name, index=False)
    return new_df

### Missing data handling
In this section, we will handle all the missing data in our data frame by: <br>
• Handle 'Fire Cause': fill the empty data with 'Unknown' <br>
• Remove rows with missing values

In [4]:
def handleMissingData(df, file_name):    
    new_df = df.copy()
    new_df["FireCause"].fillna('Unknown', inplace=True)
    new_df.dropna(inplace=True)
    new_df.to_csv(file_name, index=False)
    return new_df

### Duplicate data handling
In this section, we will handle all the duplicate data in our data frame by: <br>
• Remove duplicate fires by UniqueFireIdentifier <br>
• Keep only one copy of each duplicated rows

In [5]:
def handleDuplicateData(df, file_name):
    new_df = df.copy()
    new_df.drop_duplicates(subset=['UniqueFireIdentifier'], keep='first', inplace=True)
    new_df.drop_duplicates(keep='first', inplace=True)
    new_df.to_csv(file_name, index=False)
    return new_df

### Data type conversion
In this section, we will handle all the data type conversion in our data frame by: <br>
• Converting FireDiscoveryDateTime and FireOutDateTime to ISO format(yyyy-MM-ddTHH:mm:ss) for the weather API <br>
• Adding Fire Duration column <br>
• Convert Fire Cause to categoric column: {'Natural': 1, 'Human': 2, 'Undetermined': 3, 'Unknown':4} <br>
• Adding a new binary column: CausedByWeather (1 -> True, 0 -> False) <br>

In [6]:
# This function is getting a series with a date time type and updates it to ISO format yyyy-MM-ddTHH:mm:ss
def convertToISOFormat(series):
    series = series.str.replace(' ', 'T', regex=False)
    series = series.str.replace('/', '-', regex=False)
    series = series.str.replace('+', ':', regex=False)
    series = series.str[:-3]
    return series

In [7]:
# This function is creating a fire duration column by calculating the days between start and end date of the fires
def createFireDurationColumn(df):
    fire_duration = []
    for row in df.index:
        start_date_time = df['FireDiscoveryDateTime'][row]
        end_date_time = df['FireOutDateTime'][row]
        start_date = dt(int(start_date_time[:4]),int(start_date_time[5:7]),int(start_date_time[8:10]))
        end_date = dt(int(end_date_time[:4]),int(end_date_time[5:7]),int(end_date_time[8:10]))
        fire_duration.append((end_date - start_date).days)
    return fire_duration


In [8]:
# This function converting the Fire Cause column to categoric column
def convertFireCauseToCategoryType(df):
    df['FireCause'] = df['FireCause'].astype('category')
    replace_map = {'Natural': 1, 'Human': 2, 'Undetermined': 3, 'Unknown':4}
    df.replace(replace_map, inplace=True)

In [9]:
# This function creating a binary column CausedByWeather  (1 -> True, 0 -> False)
def createCausedByWeatherColumn(df):
    CausedByWeather = []
    for row in df.index:
        if df['FireCause'][row] == 1:
            CausedByWeather.append(1)
        else:
            CausedByWeather.append(0)
    return CausedByWeather

In [10]:
# This function is handling the data conversion
def handleDataConversion(df, file_name):
    new_df = df.copy()
    new_df['FireDiscoveryDateTime'] = convertToISOFormat(new_df['FireDiscoveryDateTime'])
    new_df['FireOutDateTime'] = convertToISOFormat(new_df['FireOutDateTime'])
    new_df['FireDuration'] = createFireDurationColumn(new_df)
    convertFireCauseToCategoryType(new_df)
    new_df['CausedByWeather'] = createCausedByWeatherColumn(new_df)
    new_df.to_csv(file_name, index=False)
    return new_df

### Outliers handling
In this section, we will handle all the outliers in our data frame by: <br>
• Analyzing the Fire Duration outliers <br>
• Analyzing the Fire Location outliers

In [11]:
# This function is handling Fire Duration Outliers by using IQR
def handleFireDurationOutliers(df):
    pd.options.mode.chained_assignment = None 
    Q1 = np.percentile(df['FireDuration'], 25)
    Q3 = np.percentile(df['FireDuration'], 75)
    IQR = Q3 - Q1
    df['FireDuration'][(df['FireDuration'] < Q1 - 1.5*IQR) | (df['FireDuration'] > Q3 + 1.5*IQR )] = np.nan
    df.dropna(inplace=True)

In [12]:
# This function is validating the Country against the latitude and longitude values and validates that the country is in USA
def validateCountryLatitdeLongitude(df):
    geolocator = Nominatim(user_agent="geoapiExercises")
    for row in df.index:
        # Get current data
        curr_state = df['POOCounty'][row]
        latitude = str(df['InitialLatitude'][row])
        longitude = str(df['InitialLongitude'][row])
        
        # Get data using geopy
        location = geolocator.reverse(latitude+","+longitude)
        address = location.raw['address']
        state = address.get('county', '')
        country = address.get('country', '')
        
        # Validation
        if not((curr_state.lower() in state.lower()) and (country.lower() == 'United States')):
            df[row] = np.nan
        
    df.dropna(inplace=True)

In [13]:
# This function is handling the outliers
def handleOutliers(df, file_name):
    new_df = df.copy()
    handleFireDurationOutliers(new_df)
    validateCountryLatitdeLongitude(new_df)
    new_df.to_csv(file_name, index=False)
    return new_df

### print function
In this section, we will implement a simple print function for the main program. <br>
We will execute this function after each cleaning type

In [14]:
def printDataFrameInfo(df):
    print("** Data frame information ** ")
    print("Shape:")
    print(df.shape)
    print("Info:")
    print(df.info())
    print("Describe:")
    print(df.describe(include='all'))

## Main Program
In this section, we will run all the data cleaning process

In [15]:
df=pd.read_csv(BASE_CSV)
print("Before cleaning:")
printDataFrameInfo(df)
df

C:\Users\nasth\AppData\Local\Temp\ipykernel_12732\1046929988.py:1: DtypeWarning: Columns (10,11,29,58,61,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(BASE_CSV)


Before cleaning:
** Data frame information ** 
Shape:
(219358, 93)
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219358 entries, 0 to 219357
Data columns (total 93 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   ABCDMisc                         10813 non-null   object 
 1   ADSPermissionState               219358 non-null  object 
 2   CalculatedAcres                  4900 non-null    float64
 3   ContainmentDateTime              131246 non-null  object 
 4   ControlDateTime                  119570 non-null  object 
 5   DailyAcres                       150112 non-null  float64
 6   DiscoveryAcres                   158154 non-null  float64
 7   DispatchCenterID                 181979 non-null  object 
 8   EstimatedCostToDate              13638 non-null   float64
 9   FinalFireReportApprovedByTitle   0 non-null       float64
 10  FinalFireReportApprovedByUnit    2616 non-null    objec

,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,...,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID
0,NaN,CERTIFIED,50.64,2020/08/06 23:13:07+00,2020/08/06 23:13:24+00,50.60,20.00,MTMCC,NaN,NaN,...,0,NaN,NaN,2020/08/06 19:50:29+00,2020/08/12 20:46:01+00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,NaN,NaN
1,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.10,CALACC,NaN,NaN,...,0,NaN,NaN,2020/02/28 20:52:36+00,2020/02/28 20:52:36+00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,NaN,NaN
2,NaN,DEFAULT,NaN,2017/10/18 00:30:00+00,2017/10/18 00:35:00+00,50.00,50.00,MTKIC,NaN,NaN,...,0,NaN,NaN,2017/10/18 13:46:40+00,2017/11/09 22:08:19+00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,NaN,NaN
3,NaN,DEFAULT,NaN,NaN,NaN,NaN,NaN,CAMVIC,NaN,NaN,...,0,NaN,NaN,2019/07/01 20:10:12+00,2019/07/01 20:10:12+00,IRWIN,{5DF06F41-9948-49D3-B00A-2D3A1D1049C5},NaN,NaN,NaN
4,NaN,DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,2016/06/20 22:39:02+00,2016/06/20 22:39:02+00,IRWIN,{F378818E-D541-4E0A-9A44-C81886C2B8B4},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219353,NaN,DEFAULT,NaN,2022/05/19 05:24:00+00,NaN,0.01,0.01,CARRCC,NaN,NaN,...,0,NaN,NaN,2022/05/19 05:34:01+00,2022/05/19 05:35:18+00,IRWIN,{54E4F816-8F07-4FED-B0F2-0002A4557577},0.0,NaN,NaN
219354,NaN,DEFAULT,NaN,2022/05/19 06:07:00+00,NaN,0.01,0.01,CARRCC,NaN,NaN,...,0,NaN,NaN,2022/05/19 05:54:05+00,2022/05/19 07:05:11+00,IRWIN,{B0811A71-78D6-45AD-A4AA-9DB841925EC1},0.0,NaN,NaN
219355,NaN,DEFAULT,NaN,2022/05/19 07:22:06+00,2022/05/19 07:57:20+00,0.01,0.01,CAMMCC,NaN,NaN,...,0,NaN,NaN,2022/05/19 07:22:16+00,2022/05/19 07:57:27+00,IRWIN,{2A5EC384-919C-4ECB-ACBC-656E118E1FB1},0.0,NaN,NaN
219356,NaN,DEFAULT,NaN,2022/05/19 07:52:01+00,2022/05/19 07:52:04+00,0.01,0.01,CAMMCC,NaN,NaN,...,0,NaN,NaN,2022/05/19 07:52:08+00,2022/05/19 07:52:13+00,IRWIN,{6661A7F0-34EF-4E23-85D1-B19F181B3C2D},0.0,NaN,NaN


In [16]:
df=removeColumns(df, COLS, CSV_COL_CLEAN)
print("After columns cleaning:")
printDataFrameInfo(df)
df

After columns cleaning:
** Data frame information ** 
Shape:
(219358, 7)
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219358 entries, 0 to 219357
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   UniqueFireIdentifier   219358 non-null  object 
 1   FireDiscoveryDateTime  219358 non-null  object 
 2   FireOutDateTime        130225 non-null  object 
 3   POOCounty              219197 non-null  object 
 4   InitialLatitude        157883 non-null  float64
 5   InitialLongitude       157883 non-null  float64
 6   FireCause              191875 non-null  object 
dtypes: float64(2), object(5)
memory usage: 11.7+ MB
None
Describe:
       UniqueFireIdentifier   FireDiscoveryDateTime         FireOutDateTime  \
count                219358                  219358                  130225   
unique               218851                  208578                   93695   
top       2014-CAHUU-003732  

,UniqueFireIdentifier,FireDiscoveryDateTime,FireOutDateTime,POOCounty,InitialLatitude,InitialLongitude,FireCause
0,2020-MTLG42-000224,2020/08/06 18:58:00+00,2020/08/12 14:00:00+00,Carter,45.784960,-104.495800,Human
1,2020-CALAC-066100,2020/02/28 20:45:40+00,NaN,Los Angeles,33.808980,-118.180700,Unknown
2,2017-MTNWS-000878,2017/10/17 20:20:24+00,2017/11/09 21:59:59+00,Flathead,48.071670,-114.830300,Human
3,2019-CAMVU-009269,2019/07/01 19:54:00+00,NaN,San Diego,NaN,NaN,NaN
4,2016-CANEU-014375,2016/06/20 22:05:59+00,NaN,Placer,NaN,NaN,NaN
...,...,...,...,...,...,...,...
219353,2022-CARRU-071648,2022/05/19 05:23:38+00,NaN,Riverside,33.568407,-116.079902,Undetermined
219354,2022-CARRU-071660,2022/05/19 05:53:56+00,NaN,Riverside,33.800675,-117.271709,Undetermined
219355,2022-CAMMU-010672,2022/05/19 07:11:38+00,NaN,Merced,37.288402,-120.457292,Undetermined
219356,2022-CAMMU-010673,2022/05/19 07:38:16+00,NaN,Madera,36.982695,-120.026844,Undetermined


In [17]:
df=handleMissingData(df, CSV_MISSING_CLEAN)
print("After missing data handling:")
printDataFrameInfo(df)
df

After missing data handling:
** Data frame information ** 
Shape:
(108097, 7)
Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 108097 entries, 0 to 219332
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   UniqueFireIdentifier   108097 non-null  object 
 1   FireDiscoveryDateTime  108097 non-null  object 
 2   FireOutDateTime        108097 non-null  object 
 3   POOCounty              108097 non-null  object 
 4   InitialLatitude        108097 non-null  float64
 5   InitialLongitude       108097 non-null  float64
 6   FireCause              108097 non-null  object 
dtypes: float64(2), object(5)
memory usage: 6.6+ MB
None
Describe:
       UniqueFireIdentifier   FireDiscoveryDateTime         FireOutDateTime  \
count                108097                  108097                  108097   
unique               108087                  104925                   79125   
top       2015-UTNES-0000

,UniqueFireIdentifier,FireDiscoveryDateTime,FireOutDateTime,POOCounty,InitialLatitude,InitialLongitude,FireCause
0,2020-MTLG42-000224,2020/08/06 18:58:00+00,2020/08/12 14:00:00+00,Carter,45.78496,-104.49580,Human
2,2017-MTNWS-000878,2017/10/17 20:20:24+00,2017/11/09 21:59:59+00,Flathead,48.07167,-114.83030,Human
5,2020-MSMNF-000308,2020/11/23 19:17:00+00,2020/11/30 14:29:59+00,Perry,31.06819,-89.06972,Human
6,2019-UTUWF-000883,2019/10/26 21:29:00+00,2019/11/13 00:14:59+00,Utah,40.07631,-111.41820,Unknown
7,2020-MTCES-006641,2020/08/27 14:06:38+00,2020/08/27 20:52:59+00,Beaverhead,44.65363,-111.56360,Natural
...,...,...,...,...,...,...,...
219231,2022-OKWSR-001355,2022/05/17 15:45:00+00,2022/05/18 10:30:00+00,Custer,35.60432,-99.27886,Human
219235,2022-MTLNF-200253,2022/05/18 15:17:59+00,2022/05/18 16:26:00+00,Missoula,46.83358,-114.10830,Human
219249,2022-AZA4S-220586,2022/05/17 23:56:51+00,2022/05/18 21:59:59+00,Maricopa,33.94848,-112.86590,Human
219278,2022-ORDEF-000157,2022/05/18 19:21:08+00,2022/05/18 21:41:00+00,Deschutes,43.77030,-121.52920,Human


In [18]:
df=handleDuplicateData(df, CSV_DUP_CLEAN)
print("After duplicate data handling:")
printDataFrameInfo(df)
df

After duplicate data handling:
** Data frame information ** 
Shape:
(108087, 7)
Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 108087 entries, 0 to 219332
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   UniqueFireIdentifier   108087 non-null  object 
 1   FireDiscoveryDateTime  108087 non-null  object 
 2   FireOutDateTime        108087 non-null  object 
 3   POOCounty              108087 non-null  object 
 4   InitialLatitude        108087 non-null  float64
 5   InitialLongitude       108087 non-null  float64
 6   FireCause              108087 non-null  object 
dtypes: float64(2), object(5)
memory usage: 6.6+ MB
None
Describe:
       UniqueFireIdentifier   FireDiscoveryDateTime         FireOutDateTime  \
count                108087                  108087                  108087   
unique               108087                  104916                   79119   
top      2020-MTLG42-00

,UniqueFireIdentifier,FireDiscoveryDateTime,FireOutDateTime,POOCounty,InitialLatitude,InitialLongitude,FireCause
0,2020-MTLG42-000224,2020/08/06 18:58:00+00,2020/08/12 14:00:00+00,Carter,45.78496,-104.49580,Human
2,2017-MTNWS-000878,2017/10/17 20:20:24+00,2017/11/09 21:59:59+00,Flathead,48.07167,-114.83030,Human
5,2020-MSMNF-000308,2020/11/23 19:17:00+00,2020/11/30 14:29:59+00,Perry,31.06819,-89.06972,Human
6,2019-UTUWF-000883,2019/10/26 21:29:00+00,2019/11/13 00:14:59+00,Utah,40.07631,-111.41820,Unknown
7,2020-MTCES-006641,2020/08/27 14:06:38+00,2020/08/27 20:52:59+00,Beaverhead,44.65363,-111.56360,Natural
...,...,...,...,...,...,...,...
219231,2022-OKWSR-001355,2022/05/17 15:45:00+00,2022/05/18 10:30:00+00,Custer,35.60432,-99.27886,Human
219235,2022-MTLNF-200253,2022/05/18 15:17:59+00,2022/05/18 16:26:00+00,Missoula,46.83358,-114.10830,Human
219249,2022-AZA4S-220586,2022/05/17 23:56:51+00,2022/05/18 21:59:59+00,Maricopa,33.94848,-112.86590,Human
219278,2022-ORDEF-000157,2022/05/18 19:21:08+00,2022/05/18 21:41:00+00,Deschutes,43.77030,-121.52920,Human


In [19]:
df=handleDataConversion(df, CSV_DATA_CONVERSION)
print("After data conversion handling:")
printDataFrameInfo(df)
df

After data conversion handling:
** Data frame information ** 
Shape:
(108087, 9)
Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 108087 entries, 0 to 219332
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   UniqueFireIdentifier   108087 non-null  object  
 1   FireDiscoveryDateTime  108087 non-null  object  
 2   FireOutDateTime        108087 non-null  object  
 3   POOCounty              108087 non-null  object  
 4   InitialLatitude        108087 non-null  float64 
 5   InitialLongitude       108087 non-null  float64 
 6   FireCause              108087 non-null  category
 7   FireDuration           108087 non-null  int64   
 8   CausedByWeather        108087 non-null  int64   
dtypes: category(1), float64(2), int64(2), object(4)
memory usage: 11.6+ MB
None
Describe:
       UniqueFireIdentifier FireDiscoveryDateTime      FireOutDateTime  \
count                108087                1

,UniqueFireIdentifier,FireDiscoveryDateTime,FireOutDateTime,POOCounty,InitialLatitude,InitialLongitude,FireCause,FireDuration,CausedByWeather
0,2020-MTLG42-000224,2020-08-06T18:58:00,2020-08-12T14:00:00,Carter,45.78496,-104.49580,2,6,0
2,2017-MTNWS-000878,2017-10-17T20:20:24,2017-11-09T21:59:59,Flathead,48.07167,-114.83030,2,23,0
5,2020-MSMNF-000308,2020-11-23T19:17:00,2020-11-30T14:29:59,Perry,31.06819,-89.06972,2,7,0
6,2019-UTUWF-000883,2019-10-26T21:29:00,2019-11-13T00:14:59,Utah,40.07631,-111.41820,4,18,0
7,2020-MTCES-006641,2020-08-27T14:06:38,2020-08-27T20:52:59,Beaverhead,44.65363,-111.56360,1,0,1
...,...,...,...,...,...,...,...,...,...
219231,2022-OKWSR-001355,2022-05-17T15:45:00,2022-05-18T10:30:00,Custer,35.60432,-99.27886,2,1,0
219235,2022-MTLNF-200253,2022-05-18T15:17:59,2022-05-18T16:26:00,Missoula,46.83358,-114.10830,2,0,0
219249,2022-AZA4S-220586,2022-05-17T23:56:51,2022-05-18T21:59:59,Maricopa,33.94848,-112.86590,2,1,0
219278,2022-ORDEF-000157,2022-05-18T19:21:08,2022-05-18T21:41:00,Deschutes,43.77030,-121.52920,2,0,0


In [20]:
df=handleOutliers(df, CSV_HANDLE_OUTLIERS)
print("After outliers data handling:")
printDataFrameInfo(df)
df

After outliers data handling:
** Data frame information ** 
Shape:
(94742, 9)
Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 94742 entries, 0 to 219332
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   UniqueFireIdentifier   94742 non-null  object  
 1   FireDiscoveryDateTime  94742 non-null  object  
 2   FireOutDateTime        94742 non-null  object  
 3   POOCounty              94742 non-null  object  
 4   InitialLatitude        94742 non-null  float64 
 5   InitialLongitude       94742 non-null  float64 
 6   FireCause              94742 non-null  category
 7   FireDuration           94742 non-null  float64 
 8   CausedByWeather        94742 non-null  int64   
dtypes: category(1), float64(3), int64(1), object(4)
memory usage: 6.6+ MB
None
Describe:
       UniqueFireIdentifier FireDiscoveryDateTime      FireOutDateTime  \
count                 94742                 94742           

,UniqueFireIdentifier,FireDiscoveryDateTime,FireOutDateTime,POOCounty,InitialLatitude,InitialLongitude,FireCause,FireDuration,CausedByWeather
0,2020-MTLG42-000224,2020-08-06T18:58:00,2020-08-12T14:00:00,Carter,45.78496,-104.49580,2,6.0,0
2,2017-MTNWS-000878,2017-10-17T20:20:24,2017-11-09T21:59:59,Flathead,48.07167,-114.83030,2,23.0,0
5,2020-MSMNF-000308,2020-11-23T19:17:00,2020-11-30T14:29:59,Perry,31.06819,-89.06972,2,7.0,0
6,2019-UTUWF-000883,2019-10-26T21:29:00,2019-11-13T00:14:59,Utah,40.07631,-111.41820,4,18.0,0
7,2020-MTCES-006641,2020-08-27T14:06:38,2020-08-27T20:52:59,Beaverhead,44.65363,-111.56360,1,0.0,1
...,...,...,...,...,...,...,...,...,...
219231,2022-OKWSR-001355,2022-05-17T15:45:00,2022-05-18T10:30:00,Custer,35.60432,-99.27886,2,1.0,0
219235,2022-MTLNF-200253,2022-05-18T15:17:59,2022-05-18T16:26:00,Missoula,46.83358,-114.10830,2,0.0,0
219249,2022-AZA4S-220586,2022-05-17T23:56:51,2022-05-18T21:59:59,Maricopa,33.94848,-112.86590,2,1.0,0
219278,2022-ORDEF-000157,2022-05-18T19:21:08,2022-05-18T21:41:00,Deschutes,43.77030,-121.52920,2,0.0,0
